# T4 Clustering: algorisme K-mitjanes

# Índex

1. Clustering particional
2. Criteri suma d'errors quadràtics
3. Algorisme $K$-mitjanes de Duda i Hart
4. Algorisme $K$-mitjanes convencional

# 1 Clustering particional

**Clustering particional:** $\;$ donat un conjunt de $N$ dades $\mathcal{D}$ i un nombre de clústers $K,\,$ el clustering particional consisteix a optimitzar alguna funció criteri $J(\Pi)$ per a avaluar la qualitat de qualsevol partició $\Pi$ de les dades en $K$ clústers
$$\Pi^* = \operatorname*{argopt}_{\Pi} J(\Pi)$$

**Intractabilitat:** $\;$ el clustering particional és en general un problema intractable ja que el nombre de particions a explorar creix exponencialment amb $N$ i $K$ (veure [nombres d'Stirling del segon tipus](https://en.wikipedia.org/wiki/Stirling_numbers_of_the_second_kind))

**Aproximació usual:** $\;$ fem ús d'algorismes aproximats per a optimitzar un criteri particular com ara la suma d'errors quadràtics

# 2 Criteri suma d'errors quadràtics

**Suma d'errors quadràtics (SEQ):** $\;$ d'una partició $\Pi=\{X_1,\dotsc,X_K\}$
$$J(\Pi) = \sum_{k=1}^K J_k%
\quad\text{amb}\quad%
J_k = \sum_{\boldsymbol{x}\in X_k} \lVert{\boldsymbol{x}-\boldsymbol{m}_k}\rVert_2^2
\quad\text{i}\quad%
\boldsymbol{m}_k = \frac{1}{\lvert X_k\rvert}\sum_{\boldsymbol{x}\in X_k} \boldsymbol{x}$$

**Interpretació:**
* Cada clúster $k$ es representa pel seu **centroide** o **mitjana** $\,\boldsymbol{m}_k$
* Si $\boldsymbol{x}$ pertany al clúster $k$, $\boldsymbol{x}-\boldsymbol{m}_k$ és el **vector error** obtés al representar $\boldsymbol{x}$ amb $\boldsymbol{m}_k$
* L'error associat a $\boldsymbol{x}$ es mesura amb la norma Euclidiana del seu vector error, $\lVert{\boldsymbol{x}-\boldsymbol{m}_k}\rVert_2$
* Anomenem **distorsió** del clúster $k$, $\,J_k,\,$ a la suma d'errors al quadrat de les seues dades
* El criteri SEQ és la suma de les distorsions de tots els clústers i, òbviament, és un criteri a minimitzar
* Idealment, esperem clústers hiper-esfèrics compactes i de grandària pareguda, al voltant de $K$ mitjanes ben separades

# 3 Algorisme $K$-mitjanes de Duda i Hart

# 4 Algorisme $K$-mitjanes convencional